In [ ]:
from active_critic.utils.gym_utils import make_vec_env, make_dummy_vec_env
import gym
from stable_baselines3 import PPO
import torch
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.evaluation import evaluate_policy
from utilsMW.dataLoaderMW import TorchDatasetMWToy
import warnings
from typing import Any, Dict, Optional, Type, Union

import numpy as np
import torch as th
from gym import spaces
from torch.nn import functional as F

from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from stable_baselines3.common.policies import ActorCriticCnnPolicy, ActorCriticPolicy, BasePolicy, MultiInputActorCriticPolicy
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import explained_variance,  get_schedule_fn

from searchTest.toyEnvironment import check_outpt
from utilsMW.dataLoaderMW import TorchDatasetMW
from torch.utils.data import DataLoader
from imitation.algorithms.adversarial import gail 
from imitation.util.networks import RunningNorm
from imitation.rewards.reward_nets import BasicRewardNet
from stable_baselines3.ppo import MlpPolicy
from active_critic.utils.tboard_graphs import TBoardGraphs
def sample_expert_transitions(expert, env, num):

    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=None, min_episodes=num),
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts), rollouts

def asd(env, learner):
    success = []
    rews = []
    for i in range(100):
        obs = env.reset()
        done = False
        while not done:
            action, _ = learner.predict(obs)
            obs, rew, done, info = env.step(action)
            rews.append(rew)
            if info[0]['success'] > 0:
                success.append(info[0]['success'])
                break
            if done:
                success.append(0)
    return np.array(success), np.array(rews)

from sb3_contrib import TQC
env, vec_expert = make_dummy_vec_env(name='pickplace', seq_len=200)
val_env, _ = make_dummy_vec_env(name='pickplace', seq_len=200)

policy_kwargs = dict(n_critics=2, n_quantiles=25)
learner = TQC("MlpPolicy", env, top_quantiles_to_drop_per_net=2, verbose=1, policy_kwargs=policy_kwargs)

tboard = TBoardGraphs(logname='TQC with expert Data', data_path='/data/bing/hendrik/gboard/')
learner.use_alt_policy = True
learner.alternative_policy = vec_expert
best_success = -1
for i in range(10000):
    learner.learn(5000, log_interval=1000)
    success, rews = asd(env=val_env, learner=learner)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=env.envs[0].reset_count)
    mean_success = success.mean()
    tboard.addTrainScalar('Success Rate', value=th.tensor(mean_success), stepid=env.envs[0].reset_count)
    print(env.envs[0].reset_count)
    print(f'Success Rate: {mean_success}')
    if mean_success > best_success:
        learner.save('/data/bing/hendrik/reach_tqc')
        learner.save_replay_buffer('/databing/hendrik/reach_tqc_rpb')


In [1]:
from sb3_contrib import TQC

tqc = TQC.load('/data/bing/hendrik/reatch_tqc.zip')

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tqc.load_replay_buffer('/data/bing/hendrik/reach_tqc_rpb.pkl')